<a href="https://colab.research.google.com/github/SamudralaAjaykumarrr/Resume-Skill-Gap-Analyzer/blob/main/Resume_Skill_Gap_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------------------------
# INSTALL ONLY SAFE PACKAGES
# ---------------------------------------------------
!pip install scikit-learn gradio==4.16.0

# ---------------------------------------------------
# IMPORTS
# ---------------------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import re
import json

# ---------------------------------------------------
# CLEANING FUNCTION
# ---------------------------------------------------
def clean_text(t):
    t = t.lower()
    t = re.sub(r"[^a-z0-9+ ]", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

# ---------------------------------------------------
# SKILL EXTRACTION (simple but effective)
# ---------------------------------------------------
def extract_skills(text):
    tech_keywords = [
        "python","java","c++","sql","nosql","mongodb","mysql","docker","kubernetes",
        "aws","gcp","azure","pytorch","tensorflow","scikit-learn","pandas","numpy",
        "data analysis","deep learning","machine learning","nlp","react","node",
        "api","linux","git","github","rest","javascript"
    ]
    found = []
    t = text.lower()
    for k in tech_keywords:
        if k in t:
            found.append(k)
    return list(set(found))

# ---------------------------------------------------
# MAIN FUNCTION
# ---------------------------------------------------
def analyze(job_desc, resume_text, top_k):
    if len(job_desc.strip()) < 20 or len(resume_text.strip()) < 20:
        return "Enter enough text.", "{}"

    job_clean = clean_text(job_desc)
    resume_clean = clean_text(resume_text)

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([job_clean, resume_clean])

    similarity = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    similarity = round(float(similarity) * 100, 2)

    job_skills = extract_skills(job_desc)
    resume_skills = extract_skills(resume_text)

    missing = [s for s in job_skills if s not in resume_skills]

    result = f"""
📝 **Resume – Job Match:** {similarity}%

**Skills in Job Description:**
{', '.join(job_skills) if job_skills else 'None found'}

**Skills in Resume:**
{', '.join(resume_skills) if resume_skills else 'None found'}

**Missing Skills (You Should Add):**
{', '.join(missing) if missing else 'No missing skills! Perfect match.'}
"""

    json_out = {
        "similarity_percent": similarity,
        "job_skills": job_skills,
        "resume_skills": resume_skills,
        "missing_skills": missing
    }

    return result, json.dumps(json_out, indent=4)

# ---------------------------------------------------
# GRADIO UI
# ---------------------------------------------------
demo = gr.Interface(
    fn=analyze,
    inputs=[
        gr.Textbox(lines=8, label="Job Description"),
        gr.Textbox(lines=8, label="Resume Text"),
        gr.Slider(3,20,value=7,step=1,label="Top-k (Not used but kept for UI consistency)")
    ],
    outputs=[
        gr.Textbox(label="Analysis"),
        gr.Textbox(label="JSON Output")
    ],
    title="Resume Skill Gap Analyzer (Stable Version)"
)

demo.launch()


IMPORTANT: You are using gradio version 4.16.0, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://662b82b610bd121793.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
